# Contexto de Negocio y Marco de Trabajo

## Empresa y situación
El flujo de pedidos crece y requiere actualizaciones frecuentes del lakehouse. Reprocesar todo a diario es costoso.

## Qué / Por qué / Para qué / Cuándo / Cómo
- Qué: Pipeline incremental de órdenes en Parquet.
- Por qué: Reducir tiempo y costos de cómputo vs recargas completas.
- Para qué: Mantener datos frescos para analítica y monitoreo operativo.
- Cuándo: Cada 1–4 horas según criticidad.
- Cómo: Checkpoint de fecha, filtrado incremental, append y actualización de checkpoint.

In [1]:
# Configuración del entorno
import sys
from pathlib import Path

# Agregar el directorio raíz al path para imports
root_dir = Path.cwd().parent.parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

print(f"✅ Root directory agregado al path: {root_dir}")

✅ Root directory agregado al path: f:\GitHub\supply-chain-data-notebooks


# Contexto de Negocio y Marco de Trabajo

## Empresa y situación
Retailer con sistema de órdenes distribuido que genera archivos diarios. Se necesita un pipeline incremental eficiente para procesar solo las órdenes nuevas o modificadas.

## Qué / Por qué / Para qué / Cuándo / Cómo
- Qué: Pipeline incremental que detecta y procesa solo las órdenes nuevas desde la última ejecución.
- Por qué: Evitar reprocesar todo el histórico diariamente ahorra tiempo y recursos computacionales.
- Para qué: Mantener un data warehouse actualizado con baja latencia para análisis near real-time.
- Cuándo: Ejecutar cada hora o cada 4 horas según la criticidad del negocio.
- Cómo: Usar checkpoints para rastrear la última fecha procesada y filtrar incrementalmente.

---
id: "DE-02"
title: "Pipeline incremental de órdenes"
specialty: "Data Engineering"
process: "Deliver"
level: "Intermediate"
tags: ["etl", "incremental", "orders", "python", "parquet"]
estimated_time_min: 45
---

import pandas as pd
from src.utils.paths import DATA_RAW, DATA_PROCESSED, ensure_dirs
from src.utils.logging import get_logger

ensure_dirs()
logger = get_logger('DE-02')
logger.info('Iniciando pipeline incremental de órdenes')
print('✅ Librerías cargadas y rutas preparadas')
---

In [2]:
# ⚙️ Preparación de entorno y rutas
# Si esta celda tarda demasiado o se cuelga:
# 1) Abre la paleta de comandos (Ctrl+Shift+P)
# 2) "Jupyter: Restart Kernel"
# 3) "Run All Above/Below" o ejecuta desde la primera celda

import sys
from pathlib import Path

# Detectar raíz del repo (buscando pyproject.toml o carpeta src)
_candidates = [Path.cwd(), *Path.cwd().parents]
_repo_root = None
for _p in _candidates:
    if (_p / 'pyproject.toml').exists() or (_p / 'src').exists():
        _repo_root = _p
        break
if _repo_root is None:
    _repo_root = Path.cwd()

if str(_repo_root) not in sys.path:
    sys.path.insert(0, str(_repo_root))

print(f"✅ Entorno listo. Raíz del repo: {_repo_root}")

✅ Entorno listo. Raíz del repo: f:\GitHub\supply-chain-data-notebooks


orders = pd.read_csv(DATA_RAW / 'orders.csv')
orders['date'] = pd.to_datetime(orders['date'])
# Filtrado incremental robusto usando Timestamp
new_orders = orders[orders['date'] > last_ts]
logger.info(f'Nuevas órdenes: {len(new_orders)}')
print(new_orders.head())
Procesar toda la historia diariamente es costoso. Un pipeline incremental reduce tiempo y recursos.

### ¿Para qué?
- Mantener actualizado el lakehouse sin recargas completas
- Habilitar near real-time analytics
- Optimizar uso de infraestructura

### ¿Cuándo?
Ejecutar cada hora o cada 4 horas según criticidad del negocio.

### ¿Cómo?
1. Leer última fecha procesada desde checkpoint
2. Filtrar órdenes con `date > last_processed`
3. Append a tabla existente
4. Actualizar checkpoint

In [ ]:
# 📚 CONCEPTO: Pipeline incremental vs carga completa (full load)
# CARGA COMPLETA (Full Load):
# - Lee TODOS los datos históricos cada ejecución
# - Simple de implementar (un solo SELECT * FROM source)
# - Costoso computacionalmente: O(N) donde N = registros totales
# - Escala mal: si tienes 100M registros, procesar 100M cada día

# CARGA INCREMENTAL (Incremental Load):
# - Lee SOLO datos nuevos/modificados desde última ejecución
# - Requiere checkpoint (marca de agua, watermark, last processed timestamp)
# - Eficiente: O(Δ) donde Δ = registros nuevos (típicamente <1% del total)
# - Escala linealmente con volumen de cambios, no con volumen total

# 💡 INTERPRETACIÓN: ¿Cuándo usar incremental?
# USAR INCREMENTAL cuando:
# - Datos históricos grandes (>1M registros)
# - Frecuencia de carga alta (cada hora, cada 15 min)
# - Modificaciones raras (append-only o updates poco frecuentes)
# - Costo de cómputo es restricción (ej: Spark clusters caros)

# NO usar incremental cuando:
# - Datos pequeños (<100K registros) → full load es suficiente
# - Lógica compleja de updates/deletes → considerar CDC (Change Data Capture)
# - Requerimientos de SCD Type 2 → necesitas comparar estados completos

# 🔍 TÉCNICA: Checkpoint persistente
# El checkpoint debe ser:
# - Persistente: guardado en disco/DB (no en memoria, se pierde al reiniciar)
# - Atómico: actualizado SOLO si carga fue exitosa (transaccionalidad)
# - Tipos comunes:
#   * Timestamp: last_processed_date (asume datos ordenados por fecha)
#   * Offset: last_record_id (para bases de datos con IDs secuenciales)
#   * Version: last_snapshot_version (para Delta Lake, Iceberg)

# 🎯 APLICACIÓN: Idempotencia en pipelines incrementales
# Un pipeline idempotente puede ejecutarse múltiples veces con mismo resultado.
# Para lograr idempotencia:
# - Deduplicar registros (usar DISTINCT o GROUP BY por clave primaria)
# - Usar UPSERT en lugar de INSERT (actualizar si existe, insertar si no)
# - Validar checkpoint antes de actualizar (evitar brechas en datos)

# ⚠️ SUPUESTO: Datos ordenados cronológicamente
# Este código asume que filtrar por date > last_checkpoint captura TODOS
# los datos nuevos. Puede fallar si:
# - Datos llegan desordenados (late-arriving data)
# - Retraso en sistema source (orden 10/01 llega después que orden 10/02)
# Solución: usar ventana de lookback (ej: last_checkpoint - 24h) o CDC

import pandas as pd
from src.utils.paths import DATA_RAW, DATA_PROCESSED, ensure_dirs
from src.utils.logging import get_logger

ensure_dirs()
logger = get_logger('DE-02')
logger.info('Iniciando pipeline incremental de órdenes')
print('✅ Librerías cargadas y rutas preparadas')


2025-12-01 16:22:49,422 - DE-02 - INFO - Iniciando pipeline incremental de órdenes


✅ Librerías cargadas y rutas preparadas
